# Extracting trips from the MATSIM results

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import os
from xml.etree import ElementTree

/home/clement/.conda/envs/covid/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Extracting public transport trips
The transport dataset contains all trips of individuals, of any mode including walking. However, we'll only consider public transports here. Hence, we'll have to filter the full dataset, which would otherwise be much too large to load in memory anyway.

In [4]:
_POPULATION_TRANSPORTS_PATH_ = "../data/extracted_data/matsim_population_transports.csv.gz"
_PUBLIC_TRANSPORTS_PATH_ = "../data/extracted_data/matsim_population_public_transports.csv.gz"

In [5]:
CHUNKSIZE = 2000000
# Since we'll write the results in 'append' mode, we need to make sure it doesn't exist already
# so that running the code will fully recreate the results
if os.path.exists(_PUBLIC_TRANSPORTS_PATH_):
    os.remove(_PUBLIC_TRANSPORTS_PATH_)

with pd.read_csv(_POPULATION_TRANSPORTS_PATH_, index_col=0, chunksize=CHUNKSIZE) as full_df:
    for k, chunk in enumerate(full_df):
        print(f"Processed {k} chunks - {k * CHUNKSIZE} rows")
        chunk = chunk[chunk['mode'] == 'pt']
        chunk.to_csv(_PUBLIC_TRANSPORTS_PATH_, mode="a", header=(k == 0))

Processed 0 chunks - 0 rows
Processed 1 chunks - 2000000 rows
Processed 2 chunks - 4000000 rows
Processed 3 chunks - 6000000 rows
Processed 4 chunks - 8000000 rows
Processed 5 chunks - 10000000 rows
Processed 6 chunks - 12000000 rows
Processed 7 chunks - 14000000 rows
Processed 8 chunks - 16000000 rows
Processed 9 chunks - 18000000 rows
Processed 10 chunks - 20000000 rows
Processed 11 chunks - 22000000 rows
Processed 12 chunks - 24000000 rows
Processed 13 chunks - 26000000 rows
Processed 14 chunks - 28000000 rows
Processed 15 chunks - 30000000 rows
Processed 16 chunks - 32000000 rows
Processed 17 chunks - 34000000 rows


# Deducing transports from links

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import os
import xmltodict
from collections import defaultdict, OrderedDict
from xml.etree import ElementTree

/home/clement/.conda/envs/covid/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Importing the schedule

Now, we'll also need the transport schedule - see https://github.com/matsim-org/matsim-code-examples/wiki/pt:

In [2]:
with open('../data/MATSIM_data/switzerland_transit_schedule.xml', 'r') as schedule_xml:
    schedule = xmltodict.parse(schedule_xml.read())['transitSchedule']

A ```transitRoute``` object contains four elements:
* A ```transportMode``` indicating the type of transport (e.g. "bus");
* A ```routeProfile```, a list of all the links that the line actually serves;
* A ```route``` element, the list of all road network links that the route goes by - we can ignore this.
* A ```departures``` list of all departure times of the route, including the actual vehicles used for each departure.

In [3]:
print(schedule.keys())
# schedule['transitLine'] is a list of transitLine objects
# whose attribute 'transitRoute' is a list of transitRoute objects.
print(schedule['transitLine'][0].keys())

odict_keys(['transitStops', 'minimalTransferTimes', 'transitLine'])
odict_keys(['@id', 'transitRoute'])


We don't really mind the lines but rather the ```transitRoute``` objects:

In [4]:
# Concatenates the lists of all transit Routes into a dict <route id> 
routes = dict()
for line in schedule['transitLine']:
    for route in line['transitRoute']:
        # Some routes are only a single string and don't contain any info, skip them
        try:
            routes[route['@id']] = route
        except:
            continue

We'll now load the public transport data that was creatred in the first section of this notebook:

In [5]:
pt_trips_df = pd.read_csv('../data/extracted_data/matsim_population_public_transports.csv.gz')
# Ignores all trips that started after midnight
pt_trips_df.loc[:, 'dep_time'] = pd.to_datetime(pt_trips_df['dep_time'], errors='coerce')
pt_trips_df = pt_trips_df[~(pt_trips_df['dep_time'].isnull())]
pt_trips_df.head()

,id,mode,dep_time,trav_time,start_link,end_link,transitRouteId,boardingTime,transitLineId,accessFacilityId,egressFacilityId
0,100000,pt,2022-10-12 13:25:17,00:04:43,912236,15050,50101_087,13:29:00,PAG_line501,8580501.link:912236,8580502.link:15050
1,1000001,pt,2022-10-12 11:45:39,00:01:21,741846,492450,07706_001,11:45:39,SBB_R_8500218-8500261,8500207.link:741846,8500206.link:492450
2,1000005,pt,2022-10-12 12:26:15,00:17:45,160222,187175,02043_024,12:39:00,BOG_line502,8578048.link:160222,8578055.link:187175
3,1000005,pt,2022-10-12 20:03:09,00:09:51,187174,160221,02038_023,20:08:00,BOG_line502,8578055.link:187174,8578048.link:160221
4,1000010,pt,2022-10-12 14:36:51,00:20:09,64987,263420,01511_004,14:45:00,SBB_IC_8501120-8506302,8500212.link:64987,8500218.link:263420


The objective is, for every public transports trip (row in the previous dataset), to find in which vehicle the trip took place. The schedule data contains the departure times of every line, as well as the vehicle associated with each departure. 
All in all, we can for every row:
* Get the data for the line (using the ```transitRouteId``` variable);
* Fetch the departure offset at the corresponding stop on the line (```accessFacilityId``` variable);
* Among all departure times of that line, select the one that is closest to the actual departure time in the simulation (variable ```dep_time``` in the dataframe). The departure time at the stop is the sum of the departure time of the line and the departure offset at that stop.
* Retrieve in the route data the vehicle id corresponding to that departure.

In [6]:
def find_vehicle(trip_row):
    """
    Given the data about a public transports trip (a row of the pt_trips_df dataframe), retrieves
    the corresponding vehicle id.uteProfile']
    """
    # Fetches the data about the associated pt route
    route_id = trip_row['transitRouteId']
    if route_id not in routes:
        return None
    line = routes[route_id]
    # Fetches the departure offset at the given stop
    # The data was directly extracted from XML so it's a bit complicated to access..
    for stop in line['routeProfile']['stop']:
        if stop['@refId'] == trip_row['accessFacilityId']:
            dep_offset = stop['@departureOffset']
    # Browses all departure times and finds the closest
    min_time_diff, vehicle = None, None
    for departure in line['departures']['departure']:
        try:
            # Sometimes the scheduled dep time is beyond 24:00 to mean that the trip started after
            # midnight. This raises an exception in to_datetime()
            schedule_dep_time = pd.to_datetime(departure['@departureTime'])
        except:
            # We ignore these departure times, as we ignored them in the trips data
            continue
        time_diff = abs(schedule_dep_time - trip_row['dep_time'])
        if min_time_diff is None or time_diff < min_time_diff:
            min_time_diff = time_diff
            vehicle = departure['@vehicleRefId']
    return vehicle

In [7]:
find_vehicle(pt_trips_df.loc[0])

'BUS_PAG_88385'

In [8]:
pt_trips_df['vehicle_id'] = pt_trips_df.apply(find_vehicle, axis=1)
pt_trips_df.head()

,id,mode,dep_time,trav_time,start_link,end_link,transitRouteId,boardingTime,transitLineId,accessFacilityId,egressFacilityId,vehicle_id
0,100000,pt,2022-10-12 13:25:17,00:04:43,912236,15050,50101_087,13:29:00,PAG_line501,8580501.link:912236,8580502.link:15050,BUS_PAG_88385
1,1000001,pt,2022-10-12 11:45:39,00:01:21,741846,492450,07706_001,11:45:39,SBB_R_8500218-8500261,8500207.link:741846,8500206.link:492450,R_SBB_04143
2,1000005,pt,2022-10-12 12:26:15,00:17:45,160222,187175,02043_024,12:39:00,BOG_line502,8578048.link:160222,8578055.link:187175,NFB_BOG_65128
3,1000005,pt,2022-10-12 20:03:09,00:09:51,187174,160221,02038_023,20:08:00,BOG_line502,8578055.link:187174,8578048.link:160221,NFB_BSU_157586
4,1000010,pt,2022-10-12 14:36:51,00:20:09,64987,263420,01511_004,14:45:00,SBB_IC_8501120-8506302,8500212.link:64987,8500218.link:263420,IC_SBB_02500


Some vehicles could not be found because the route id specified in the dataset cannot be found in the schedule. Let's count those lines:

In [13]:
not_found_count = pt_trips_df['vehicle_id'].isna().sum()
print(f"Not found: {not_found_count} out of {pt_trips_df.shape[0]} ({(not_found_count / pt_trips_df.shape[0]):1.2f}%)")

Not found: 403019 out of 5444510 (0.07%)


Let's save the results:

In [14]:
pt_trips_df.to_csv('../data/extracted_data/pt_with_vehicles.csv.gz', index=False)

In [12]:
CHUNKSIZE = 100000
with pd.read_csv(_POPULATION_TRANSPORTS_PATH_, index_col=0, chunksize=CHUNKSIZE) as full_df:
    # Modes of transportation that will be discarded
    ignored_modes = set(['walk', 'bike'])
    for k, chunk in enumerate(full_df):
        print(f"Processed {k} chunks - {k * CHUNKSIZE} rows")
        # Discard ignored transportation modes
        chunk = chunk[~(chunk['mode'].isin(ignored_modes))]
        # Converts the dep_time column to the datetime data type
        # Also sets starting time beyond midnight to NaT and then removes them
        chunk['start_time'] = pd.to_datetime(chunk['dep_time'], errors='coerce')
        chunk = chunk[~(chunk['start_time'].isnull())]
        # Converts the trav_time col to the timedelta data type and computes the activity's end time
        chunk['end_time'] = chunk['start_time'] + pd.to_timedelta(chunk['trav_time'])
        

Processed 0 chunks - 0 rows
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49773 entries, 10 to 1020083
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   mode              49773 non-null  object        
 1   dep_time          49773 non-null  object        
 2   trav_time         49773 non-null  object        
 3   start_link        49773 non-null  object        
 4   end_link          49773 non-null  object        
 5   transitRouteId    15197 non-null  object        
 6   boardingTime      15197 non-null  object        
 7   transitLineId     15197 non-null  object        
 8   accessFacilityId  15197 non-null  object        
 9   egressFacilityId  15197 non-null  object        
 10  start_time        49773 non-null  datetime64[ns]
 11  end_time          49773 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(10)
memory usage: 4.9+ MB
None


/tmp/ipykernel_13790/1601054444.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk.loc[:, 'start_time'] = pd.to_datetime(chunk['dep_time'], errors='coerce')
